In [1]:
from jupyter_dash import JupyterDash
#import dash
from dash import dcc
from dash import html
import pandas as pd

# Requirenments

Description: Emission dashboard to explore individal countries contritions, and identify the most polluting nations over time.

- Graph 1: Shows all countries by population and GDP. Date is adjustable with a slider.  
- Graph 2: Shows all time emission values for the selected country from Graph 1. Can select any or all of the three types with a dropdown. There should be an option to select total or per capita values.  
- Graph 3: Shows the top 3 most pollutant nations over time.

# Read combined dataset from the World Bank

Contains data in the following categories:

- Country stats:
  - Population, total
  - GDP per capita (constant 2015 US$)  
    
    
- Emission data:
  - CO2 emissions (kt)
  - CO2 emissions (metric tons per capita)
  - Methane emissions (kt of CO2 equivalent)
  - Nitrous oxide emissions (thousand metric tons of CO2 equivalent)

In [4]:
data_path = 'Data Sources/'
df = pd.read_csv(data_path + 'World Development Indicators.csv')
df_emissions = df.drop(columns=['Country Code', 'Series Code']).dropna()
df_emissions = df_emissions.melt(id_vars = ['Country Name', 'Series Name'], var_name = 'Year', value_name = 'Value')
df_emissions['Year'] = df_emissions['Year'].str[0:4]

In [6]:
df_emissions

,Country Name,Series Name,Year,Value
0,Afghanistan,"Population, total",1960,8622466
1,Afghanistan,GDP per capita (constant 2015 US$),1960,..
2,Afghanistan,CO2 emissions (kt),1960,..
3,Afghanistan,CO2 emissions (metric tons per capita),1960,..
4,Afghanistan,Methane emissions (kt of CO2 equivalent),1960,..
...,...,...,...,...
98947,World,GDP per capita (constant 2015 US$),2021,11010.1625910995
98948,World,CO2 emissions (kt),2021,..
98949,World,CO2 emissions (metric tons per capita),2021,..
98950,World,Methane emissions (kt of CO2 equivalent),2021,..


In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server

In [5]:
app.layout = html.Div([
                html.Div([
                    html.Div([
                        dcc.Dropdown(
                            id='crossfilter-xaxis-column',
                            options=[{'label': i, 'value': i} for i in available_indicators],
                            value='Fertility rate, total (births per woman)'
                        ),
                        dcc.RadioItems(
                            id='crossfilter-xaxis-type',
                            options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                            value='Linear',
                            labelStyle={'display': 'inline-block'}
                        )
                    ],
                    style={'width': '49%', 'display': 'inline-block'}),

                    html.Div([
                        dcc.Dropdown(
                            id='crossfilter-yaxis-column',
                            options=[{'label': i, 'value': i} for i in available_indicators],
                            value='Life expectancy at birth, total (years)'
                        ),
                        dcc.RadioItems(
                            id='crossfilter-yaxis-type',
                            options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                            value='Linear',
                            labelStyle={'display': 'inline-block'}
                        )
                    ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})

                ], style={
                    'borderBottom': 'thin lightgrey solid',
                    'backgroundColor': 'rgb(250, 250, 250)',
                    'padding': '10px 5px'
                    }
                ),
                
                html.Div([
                    dcc.Graph(
                        id='crossfilter-indicator-scatter',
                        #hoverData={'points': [{'customdata': 'Japan'}]}
                        clickData={'points': [{'customdata': 'Japan'}]}
                        #figure=update_graph("Inflation, GDP deflator (annual %)", "CO2 emissions (metric tons per capita)", "Linear", "Log", 2007)
                    )
                ], style={'width': '45%', 'display': 'inline-block', 'padding': '0 20'}),
    
                html.Div([
                    dcc.Graph(id='x-time-series'),
                    dcc.Graph(id='y-time-series'),
                ], style={'display': 'inline-block', 'width': '45%'}),
    
                html.Div(dcc.Slider(
                    id='crossfilter-year--slider',
                    min=df['Year'].min(),
                    max=df['Year'].max(),
                    value=df['Year'].max(),
                    marks={str(year): str(year) for year in df['Year'].unique()},
                    step=None
                ), style={'width': '100%', 'padding': '0px 20px 20px 20px'})

            ]
            )

In [6]:
#update_graph("Inflation, GDP deflator (annual %)", "CO2 emissions (metric tons per capita)", "Linear", "Log", 2007)
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')]
)
def update_graph(xaxis_column_name, yaxis_column_name, xaxis_type, yaxis_type, year_value):
    dff = df[df['Year'] == year_value]
    
    return {
        
        'data': [dict(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

In [7]:
def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

In [8]:
@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'clickData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '{}{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'clickData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

In [9]:
app.run_server(mode='inline')

In [10]:
final_page = app.run_server()

Dash app running on http://127.0.0.1:8050/
